In [28]:
import pandas as pd
import itertools
import numpy as np
import openpyxl


In [29]:
fuel_dict = {
    1: (0.18, 0.09, 'fossil'),      # MGO
    2: (0.15, 0.07, 'fossil'),      # LNG
    3: (0.47, 0.26, 'green_fuels'), # GREEN METHANOL
    4: (0.41, 0.22, 'green_fuels'), # GREEN NH3
    5: (0.28, 0.15, 'bio'),         # B30
    6: (0.24, 0.13, 'bio'),         # B20
    7: (0.23, 0.12, 'bio'),         # B15
    8: (0.20, 0.11, 'bio'),         # B70
    9: (0.43, 0.18, 'green_fuels')  # Helelektrisk
}

t2_values = range(2,6)
w_values = range(1,11)
r_values = range(1,1819)

fuel_price_csv = 'fuel_price2_1.csv'




In [30]:
# # Leser inn nødvendig MJ på å kjøre hver rute
mj_route = pd.read_csv('../route_generation/generated_datafiles/mj_route.csv')
mj_route_el = pd.read_csv('../route_generation/generated_datafiles/mj_route_strøm.csv')

# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
distance_dict = mj_route['mj_route'].to_dict()
#distance_dict = distance_dict['mj_route']

# lager egne distanser for elektrisk

specific_distance_dict = mj_route_el['mj_route'].to_dict()
#specific_distance_dict= specific_distance_dict['mj_route']
electric_distance_dict = {key + 1: value for key, value in specific_distance_dict.items()}

#print(distance_dict)
distance_dict = {key + 1: value for key, value in distance_dict.items()}
print(distance_dict)
mj_route

print(electric_distance_dict)

{1: 221734.4, 2: 194299.6, 3: 199485.2, 4: 178969.6, 5: 202378.0, 6: 206869.2, 7: 208123.2, 8: 210389.2, 9: 214603.2, 10: 181976.4, 11: 220056.4, 12: 188651.2, 13: 199334.4, 14: 199588.0, 15: 213814.4, 16: 200709.2, 17: 191107.6, 18: 186464.4, 19: 190129.2, 20: 200076.8, 21: 219008.4, 22: 211242.0, 23: 199803.2, 24: 203961.6, 25: 199638.4, 26: 222494.8, 27: 237840.0, 28: 243280.0, 29: 215862.0, 30: 233205.2, 31: 226078.8, 32: 235025.2, 33: 229680.0, 34: 264332.8, 35: 260332.8, 36: 232400.0, 37: 271920.0, 38: 243326.4, 39: 239769.6, 40: 248589.6, 41: 236811.6, 42: 254249.6, 43: 262572.8, 44: 216331.2, 45: 207699.6, 46: 244649.6, 47: 246031.6, 48: 266254.8, 49: 256968.4, 50: 253983.2, 51: 242809.6, 52: 247596.4, 53: 242084.8, 54: 255791.6, 55: 275694.8, 56: 265288.4, 57: 266943.2, 58: 255689.6, 59: 253916.4, 60: 247444.8, 61: 224876.4, 62: 262876.4, 63: 233029.2, 64: 238690.4, 65: 238530.4, 66: 225476.8, 67: 216510.4, 68: 239536.8, 69: 232751.6, 70: 209230.0, 71: 205546.8, 72: 177680.0, 

In [31]:
# def generate_fuel_prices(scenario_range, period_range, fuel_dict):
#     num_scenarios = len(scenario_range)
#     num_periods = len(period_range)
#     num_fuels = len(fuel_dict)
    
#     # Generate random scaling factors for each fuel, scenario, and time period
#     #scaling_factors = np.random.triangular(left=0, mode=0.5, right=1, size=(num_fuels, num_scenarios, num_periods))
#     # Adjust the scaling factors based on correlation
#     fuel_prices = np.zeros((num_fuels, num_periods, num_scenarios))

#     group_scaling_factors = {}
    
#     for _, (_, _, group) in fuel_dict.items():
#         # print(group)
#         group_scaling_factors[group] = np.random.triangular(left=0, mode=0.5, right=1, size=(num_scenarios, num_periods))
    
#         # Assuming group_scaling_factors is already defined and populated
#     # Printing scaling factors for each group
#     # for group, scaling_factors in group_scaling_factors.items():
#     #     print(f"Group: {group}")
#     #     for s_idx, scenario_scaling_factors in enumerate(scaling_factors):
#     #         for p_idx, scaling_factor in enumerate(scenario_scaling_factors):
#     #             print(f"Scenario {s_idx + 1}, Period {p_idx + 1}: {scaling_factor}")
    
    


#     for i, (fuel_key, (lower_bound, upper_bound, group)) in enumerate(fuel_dict.items()):
#         # Determine the group index based on the fuel's group
#         for s_idx, scenario in enumerate(scenario_range):
#             for p_idx, period in enumerate(period_range):
#                 # Obtain the scaling factor for the current fuel, scenario, and time period
#                 #scaling_factor = scaling_factors[i, s_idx, p_idx]
#                 # Calculate the fuel price based on the triangular distribution bounds and scaling factor
#                 fuel_price = lower_bound + (upper_bound - lower_bound) * group_scaling_factors[group][s_idx][p_idx]

#                 # Adjust fuel price based on previous period's price (if applicable)
#                 # Store the calculated fuel price
#                 fuel_prices[i, p_idx, s_idx] = fuel_price
    
    

    
#     return fuel_prices

def generate_fuel_prices(scenario_range, period_range, fuel_dict):
    num_scenarios = len(scenario_range)
    num_periods = len(period_range)
    num_fuels = len(fuel_dict)
    
    # Generate random scaling factors for each fuel, scenario, and time period
    #scaling_factors = np.random.triangular(left=0, mode=0.5, right=1, size=(num_fuels, num_scenarios, num_periods))
    # Adjust the scaling factors based on correlation
    fuel_prices = np.zeros((num_fuels, num_periods, num_scenarios))

    group_scaling_factors = {}
    
    for _, (_, _, group) in fuel_dict.items():
        group_scaling_factors[group] = np.random.triangular(left=0, mode=0.5, right=1, size=(num_scenarios, num_periods))
    
    for i, (fuel_key, (lower_bound, upper_bound, group)) in enumerate(fuel_dict.items()):
        for s_idx, scenario in enumerate(scenario_range):
            for p_idx, period in enumerate(period_range):
                fuel_price = lower_bound + (upper_bound - lower_bound) * group_scaling_factors[group][s_idx][p_idx]
                fuel_prices[i, p_idx, s_idx] = fuel_price
    
    
    
    return fuel_prices

In [32]:
# Generate fuel prices

fuel_prices = generate_fuel_prices(w_values, t2_values, fuel_dict)

# Convert fuel prices to a DataFrame
# fuel_prices_df = pd.DataFrame(columns=['F', 'R', 'T', 'O', 'Value'])
# for fuel_key, (lower_bound, upper_bound, _) in fuel_dict.items():
#     fuel_idx = list(fuel_dict.keys()).index(fuel_key)
#     for s_idx, scenario in enumerate(w_values):
#         for p_idx, period in enumerate(t2_values):
#             for route in r_values:
#                 distance = electric_distance_dict[route] if fuel_key == 6 else distance_dict[route]
#                 fuel_prices_df = pd.concat([fuel_prices_df, pd.DataFrame({
#                     'F': [fuel_key],
#                     'R': [route],
#                     'T': [period],
#                     'S': [scenario],
#                     'Value': [fuel_prices[fuel_idx, p_idx, s_idx] * distance]
#                 })], ignore_index=True)

# Convert fuel prices to a list of dictionaries
fuel_prices_data = []

for fuel_key, (lower_bound, upper_bound, _) in fuel_dict.items():
    fuel_idx = list(fuel_dict.keys()).index(fuel_key)
    for s_idx, scenario in enumerate(w_values):
        for p_idx, period in enumerate(t2_values):
            for route in r_values:
                distance = electric_distance_dict[route] if fuel_key == 6 else distance_dict[route]
                fuel_prices_data.append({
                    'F': fuel_key,
                    'R': route,
                    'T': period,
                    'S': scenario,
                    'Value': fuel_prices[fuel_idx, p_idx, s_idx] * distance
                })

# Create DataFrame from the list of dictionaries
fuel_prices_df = pd.DataFrame(fuel_prices_data)



print(fuel_prices_df)

fuel_prices_df.to_csv(fuel_price_csv, index=False)

        F     R  T   S         Value
0       1     1  2   1  26096.703695
1       1     2  2   1  22867.805308
2       1     3  2   1  23478.116864
3       1     4  2   1  21063.563532
4       1     5  2   1  23818.580700
...    ..   ... ..  ..           ...
654475  9  1814  5  10  63907.405801
654476  9  1815  5  10  62206.123976
654477  9  1816  5  10  63790.440230
654478  9  1817  5  10  59801.190581
654479  9  1818  5  10  56377.404917

[654480 rows x 5 columns]
